In [17]:


#**************** IMPORT PACKAGES ********************
from flask import Flask, render_template, request, flash, redirect, url_for
from flask_mail import Mail, Message
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import requests
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math, random
from datetime import datetime
import datetime as dt
import yfinance as yf

import re
from sklearn.linear_model import LinearRegression
from textblob import TextBlob
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#************ FLASK *****************
app = Flask(__name__)


# Configure Flask-Mail settings
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'itlearnix@gmail.com'
app.config['MAIL_PASSWORD'] = 'veyqttrtaoahxsga'
app.config['MAIL_DEFAULT_SENDER'] = 'itlearnix@gmail.com'

# Configure the OAuth instances
mail = Mail(app)

#To control caching so as to save and retrieve plot figs on client side
@app.after_request
def add_header(response):
    response.headers['Pragma'] = 'no-cache'
    response.headers['Cache-Control'] = 'no-cache, no-store, must-revalidate'
    response.headers['Expires'] = '0'
    return response

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/stock-predictions')
def stock_predictions():
    return render_template('stock-predictions.html')

@app.route('/coinvert')
def coinvert():
    return render_template('coinvert.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/contact')
def contact():
    return render_template('contact.html')

@app.route('/contact-form', methods=['POST'])
def contact_form():
    if request.method == 'POST':
        name = request.form.get('name')
        email = request.form.get('email')
        subject = request.form.get('subject')
        message = request.form.get('message')

        # Create the email message for team member
        team_subject = 'New Contact Form Submission'
        team_body = f"Name: {name}\nEmail: {email}\nSubject: {subject}\nMessage: {message}"
        team_recipients = ['yadavnikhilrao@gmail.com']

        # Send email to team member
        team_msg = Message(subject=team_subject, body=team_body, recipients=team_recipients)
        mail.send(team_msg)

        return render_template('message-sent.html')
    

@app.route('/result', methods=['POST'])
def result():
    symbol = request.form['symbol']

    # **************** FUNCTIONS TO FETCH DATA ***************************
    
    def get_historical(quote):
        end = datetime.now()
        start = datetime(end.year - 2, end.month, end.day)
        data = yf.download(quote, start=start, end=end)
        df = pd.DataFrame(data=data)
        df.to_csv('' + quote + '.csv')
        if df.empty:
            ts = TimeSeries(key='N6A6QT6IBFJOPJ70', output_format='pandas')
            data, meta_data = ts.get_daily_adjusted(symbol='NSE:' + quote, outputsize='full')
            # Format df
            # Last 2 yrs rows => 502, in ascending order => ::-1
            data = data.head(503).iloc[::-1]
            data = data.reset_index()
            # Keep Required cols only
            df = pd.DataFrame()
            df['Date'] = data['date']
            df['Open'] = data['1. open']
            df['High'] = data['2. high']
            df['Low'] = data['3. low']
            df['Close'] = data['4. close']
            df['Adj Close'] = data['5. adjusted close']
            df['Volume'] = data['6. volume']
            df.to_csv('' + quote + '.csv', index=False)
        return

    # ******************** ARIMA SECTION ********************
    
    # Define the parser function
    def parser(x):
        return datetime.strptime(x, '%Y-%m-%d')

    # ************* ARIMA SECTION ********************
    
    def ARIMA_ALGO(df):
        uniqueVals = df["Code"].unique()
        len(uniqueVals)
        df = df.set_index("Code")

        def arima_model(train, test):
            history = [x for x in train]
            predictions = list()
            for t in range(len(test)):
                model = ARIMA(history, order=(6, 1, 0)).fit()
                output = model.forecast()
                yhat = output[0]
                predictions.append(yhat)
                obs = test[t]
                history.append(obs)
            return predictions

        for company in uniqueVals[:10]:
            data = (df.loc[company, :]).reset_index()
            data['Price'] = data['Close']
            Quantity_date = data[['Price', 'Date']]
            Quantity_date.index = Quantity_date['Date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
            Quantity_date['Price'] = Quantity_date['Price'].map(lambda x: float(x))
            Quantity_date = Quantity_date.fillna(Quantity_date.bfill())
            Quantity_date = Quantity_date.drop(['Date'], axis=1)

            fig = plt.figure(figsize=(10.2, 6), dpi=65)
            plt.plot(Quantity_date)
            plt.savefig('static/Trends.png')
            plt.close(fig)

            quantity = Quantity_date.values
            size = int(len(quantity) * 0.80)
            train, test = quantity[0:size], quantity[size:len(quantity)]

            predictions = arima_model(train, test)

            fig = plt.figure(figsize=(10.2, 6), dpi=65)
            plt.plot(test, label='Actual Price')
            plt.plot(predictions, label='Predicted Price')
            plt.legend(loc=4)
            plt.savefig('static/ARIMA.png')
            plt.close(fig)
            print()
            arima_pred=predictions[-2]
            print("Tomorrow's",quote," Closing Price Prediction by ARIMA:",arima_pred)
            #rmse calculation
            error_arima = math.sqrt(mean_squared_error(test, predictions))
            print("ARIMA RMSE:",error_arima)
            return arima_pred, error_arima

    # ************* LSTM SECTION **********************

    def LSTM_ALGO(df):
        dataset_train = df.iloc[0:int(0.8 * len(df)), :]
        dataset_test = df.iloc[int(0.8 * len(df)):, :]
        training_set = dataset_train.iloc[:, 4:5].values

        sc = MinMaxScaler(feature_range=(0, 1))
        training_set_scaled = sc.fit_transform(training_set)

        X_train = []
        y_train = []
        for i in range(7, len(training_set_scaled)):
            X_train.append(training_set_scaled[i - 7:i, 0])
            y_train.append(training_set_scaled[i, 0])
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        X_forecast = np.append(X_train[-1, 1:], y_train[-1])
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_forecast = np.reshape(X_forecast, (1, X_forecast.shape[0], 1))

        regressor = Sequential()
        regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
        regressor.add(Dropout(0.1))
        regressor.add(LSTM(units=50, return_sequences=True))
        regressor.add(Dropout(0.1))
        regressor.add(LSTM(units=50, return_sequences=True))
        regressor.add(Dropout(0.1))
        regressor.add(LSTM(units=50))
        regressor.add(Dropout(0.1))
        regressor.add(Dense(units=1))

        regressor.compile(optimizer='adam', loss='mean_squared_error')
        regressor.fit(X_train, y_train, epochs=25, batch_size=32)

        real_stock_price = dataset_test.iloc[:, 4:5].values

        dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis=0)
        testing_set = dataset_total[len(dataset_total) - len(dataset_test) - 7:].values
        testing_set = testing_set.reshape(-1, 1)

        testing_set = sc.transform(testing_set)

        X_test = []
        for i in range(7, len(testing_set)):
            X_test.append(testing_set[i - 7:i, 0])
        X_test = np.array(X_test)
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

        predicted_stock_price = regressor.predict(X_test)
        predicted_stock_price = sc.inverse_transform(predicted_stock_price)

        fig = plt.figure(figsize=(10.2, 6), dpi=65)
        plt.plot(real_stock_price, label='Actual Price')
        plt.plot(predicted_stock_price, label='Predicted Price')
        plt.legend(loc=4)
        plt.savefig('static/LSTM.png')
        plt.close(fig)

        error_lstm = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))

        forecasted_stock_price = regressor.predict(X_forecast)
        forecasted_stock_price = sc.inverse_transform(forecasted_stock_price)

        lstm_pred = forecasted_stock_price[0, 0]
        print()
        print("Tomorrow's", quote, "Closing Price Prediction by LSTM:", lstm_pred)
        print("LSTM RMSE:", error_lstm)
        return lstm_pred, error_lstm

    
    #***************** LINEAR REGRESSION SECTION ******************  
    
    def LIN_REG_ALGO(df):
        #No of days to be forcasted in future
        forecast_out = int(7)
        #Price after n days
        df['Close after n days'] = df['Close'].shift(-forecast_out)
        #New df with only relevant data
        df_new=df[['Close','Close after n days']]

        #Structure data for train, test & forecast
        #lables of known data, discard last 35 rows
        y =np.array(df_new.iloc[:-forecast_out,-1])
        y=np.reshape(y, (-1,1))
        #all cols of known data except lables, discard last 35 rows
        X=np.array(df_new.iloc[:-forecast_out,0:-1])
        #Unknown, X to be forecasted
        X_to_be_forecasted=np.array(df_new.iloc[-forecast_out:,0:-1])
        
        #Traning, testing to plot graphs, check accuracy
        X_train=X[0:int(0.8*len(df)),:]
        X_test=X[int(0.8*len(df)):,:]
        y_train=y[0:int(0.8*len(df)),:]
        y_test=y[int(0.8*len(df)):,:]
        
        # Feature Scaling===Normalization
        from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        X_to_be_forecasted=sc.transform(X_to_be_forecasted)
        
        #Training
        clf = LinearRegression(n_jobs=-1)
        clf.fit(X_train, y_train)
        
        #Testing
        y_test_pred=clf.predict(X_test)
        y_test_pred=y_test_pred*(1.04)
        import matplotlib.pyplot as plt2
        fig = plt2.figure(figsize=(10.2, 6), dpi=65)
        plt2.plot(y_test,label='Actual Price' )
        plt2.plot(y_test_pred,label='Predicted Price')
        
        plt2.legend(loc=4)
        plt2.savefig('static/LR.png')
        plt2.close(fig)
        
        error_lr = math.sqrt(mean_squared_error(y_test, y_test_pred))
        
        #Forecasting
        forecast_set = clf.predict(X_to_be_forecasted)
        forecast_set=forecast_set*(1.04)
        mean=forecast_set.mean()
        lr_pred=forecast_set[0,0]
        print()
        print("Tomorrow's ",quote," Closing Price Prediction by Linear Regression: ",lr_pred)
        print("Linear Regression RMSE:",error_lr)
        return df, lr_pred, forecast_set, mean, error_lr
    
    

    #**************** SENTIMENT ANALYSIS **************************
    
    def retrieve_news_sentiment(symbol, api_key):
        
        # Define the URL for the news API
        news_api_url = f'https://newsapi.org/v2/everything?q={symbol} stock&apiKey={api_key}'

        try:
            # Send an HTTP GET request to the news API
            response = requests.get(news_api_url)

            if response.status_code == 200:
                # Parse the JSON response
                news_data = response.json()

                # Check if news articles are available
                if 'articles' in news_data and len(news_data['articles']) > 0:
                    news_list = []  # List of news headlines alongside polarity
                    global_polarity = 0  # Polarity of all news headlines
                    news_headlines_list = []  # List of news headlines only
                    pos = 0  # Num of positive headlines
                    neg = 0  # Num of negative headlines
                    neutral = 0  # Num of neutral headlines

                    for article in news_data['articles']:
                        news_text = article['title']

                        # Analyze the sentiment of the news headline
                        analysis = TextBlob(news_text)
                        polarity = analysis.sentiment.polarity
                        global_polarity += polarity

                        # Categorize the sentiment of the news headline
                        if polarity > 0:
                            sentiment = "Positive"
                            pos += 1
                        elif polarity < 0:
                            sentiment = "Negative"
                            neg += 1
                        else:
                            sentiment = "Neutral"
                            neutral += 1

                        news_list.append((news_text, sentiment))
                        news_headlines_list.append(news_text)

                    # Determine the overall sentiment based on global_polarity
                    if global_polarity > 0:
                        overall_sentiment = "Overall Positive"
                    elif global_polarity < 0:
                        overall_sentiment = "Overall Negative"
                    else:
                        overall_sentiment = "Overall Neutral"

                    # Create a pie chart for sentiment analysis
                    sentiment_data = pd.DataFrame({'Sentiment': ['Positive', 'Negative', 'Neutral'],
                                               'Count': [pos, neg, neutral]})
                    plt.figure(figsize=(4.9, 4.9))
                    plt.pie(sentiment_data['Count'], labels=sentiment_data['Sentiment'], autopct='%1.1f%%', startangle=100)
                    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

                    # Save the figure as 'static/SA.png'
                    plt.savefig('static/SA.png')

                    # Close the figure
                    plt.close()

                    return global_polarity, news_list, overall_sentiment, pos, neg, neutral
                else:
                    return 0, [], "No News Found", 0, 0, 0
            else:
                return 0, [], f"Failed to Fetch News (Status Code: {response.status_code})", 0, 0, 0

        except Exception as e:
            return 0, [], f"Error: {str(e)}", 0, 0, 0

        
    def recommending(df, global_polarity,today_stock,mean):
        if today_stock.iloc[-1]['Close'] < mean:
            if global_polarity > 0:
                idea="RISE"
                decision="BUY"
                print()
                print("##############################################################################")
                print("According to the ML Predictions and Sentiment Analysis of News, a",idea,"in",quote,"stock is expected => ",decision)
            elif global_polarity <= 0:
                idea="FALL"
                decision="SELL"
                print()
                print("According to the ML Predictions and Sentiment Analysis of News, a",idea,"in",quote,"stock is expected => ",decision)
        else:
            idea="FALL"
            decision="SELL"
            print()
            print("According to the ML Predictions and Sentiment Analysis of News, a",idea,"in",quote,"stock is expected => ",decision)
        return idea, decision





    #**************GET DATA ***************************************
    
    quote=symbol
    
    #Try-except to check if valid stock symbol
    try:
        get_historical(quote)
    except:
        return render_template('stock-predictions.html',msg="Stock Symbol Not Found. Please Enter a Valid Stock Symbol")
    else:
    
        #************** PREPROCESSUNG ***********************
        
        df = pd.read_csv(''+quote+'.csv')
        print("Today's",quote,"Stock Data: ")
        today_stock=df.iloc[-1:]
        print(today_stock)
        df = df.dropna()
        code_list=[]
        for i in range(0,len(df)):
            code_list.append(quote)
        df2=pd.DataFrame(code_list,columns=['Code'])
        df2 = pd.concat([df2, df], axis=1)
        df=df2


        arima_pred, error_arima=ARIMA_ALGO(df)
        lstm_pred, error_lstm=LSTM_ALGO(df)
        df, lr_pred, forecast_set,mean,error_lr=LIN_REG_ALGO(df)
        
        # Assuming you have an 'api_key' variable defined with your API key
        api_key = '15a2cd458fe340a78d812cc5b50ef6bf'
        #symbol = quote

        # Call the retrieve_news_sentiment function with the 'api_key' argument
        polarity, news_list, overall_sentiment, pos, neg, neutral = retrieve_news_sentiment(quote, api_key)
        
        idea, decision=recommending(df, polarity,today_stock,mean)
        print()
        print("Forecasted Prices for Next 7 days:")
        print(forecast_set)
        today_stock=today_stock.round(2)
        return render_template('result.html',quote=quote,arima_pred=round(arima_pred,2),lstm_pred=round(lstm_pred,2),
                               lr_pred=round(lr_pred,2),open_s=today_stock['Open'].to_string(index=False),
                               close_s=today_stock['Close'].to_string(index=False),adj_close=today_stock['Adj Close'].to_string(index=False),
                               news_list=news_list,overall_sentiment=overall_sentiment,idea=idea,decision=decision,high_s=today_stock['High'].to_string(index=False),
                               low_s=today_stock['Low'].to_string(index=False),vol=today_stock['Volume'].to_string(index=False),
                               forecast_set=forecast_set,error_lr=round(error_lr,2),error_lstm=round(error_lstm,2),error_arima=round(error_arima,2))

    

if __name__ == '__main__':
    app.run()
   



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/vendors.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/animate.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/magnific-popup.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/flaticon.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/slick.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/default.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/style1.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/css/responsive.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Oct/2023 23:52:30] "GET /static/assets/img/loader/load

In [4]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/e6/b3/388ab967a387cc92926f70e97688dd9a7189b29a0773db815ffc5289e2b5/yfinance-0.2.31-py2.py3-none-any.whl.metadata
  Using cached yfinance-0.2.31-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
     ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.9 MB 445.2 kB/s eta 0:00:07
     - -------------------------------------- 0.1/2.9 MB 939.4 kB/s eta 0:00:04
     --- ------------------------------------ 0.2/2.9 MB 1.6 MB/s eta 0:00:02
     ----- ---------------------------------- 0.4/2.9 MB 2.0 MB/s eta 0:00:02
     --------- ------------------------------ 0.7/2.9 MB 2.8 MB/s eta 0:00:01
     ---------- ----------------------------- 0.8/2.9 MB 3.1 MB/s eta 0:00:01
     ----------------- ---------------------- 1.3/

In [5]:
pip install flask_mail


     ---------------------------------------- 0.0/45.7 kB ? eta -:--:--
     -------- ------------------------------- 10.2/45.7 kB ? eta -:--:--
     ----------------- -------------------- 20.5/45.7 kB 217.9 kB/s eta 0:00:01
     ---------------------------------- --- 41.0/45.7 kB 279.3 kB/s eta 0:00:01
     -------------------------------------- 45.7/45.7 kB 251.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for blinker from https://files.pythonhosted.org/packages/bf/2b/11bcedb7dee4923253a4a21bae3be854bcc4f06295bd827756352016d97c/blinker-1.6.3-py3-none-any.whl.metadata
  Using cached blinker-1.6.3-py3-none-any.whl.metadata (1.9 kB)
Using cached blinker-1.6.3-py3-none-any.whl (13 kB)
  Created wheel for flask_mail: filename=Flask_Mail-0.9.1-py3-none-any.whl size=7592 sha256=6df4a091acded066b599ef6c851ae5b3c6a6a402ac4d533581a37cc01f09b6b8
  Stored in directory: c:\users\dell\appd

In [7]:
pip install alpha_vantage

  Using cached alpha_vantage-2.3.1-py3-none-any.whl (31 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install keras


  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/fe/58/34d4d8f1aa11120c2d36d7ad27d0526164b1a8ae45990a2fede31d0e59bf/keras-2.14.0-py3-none-any.whl.metadata
  Using cached keras-2.14.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)


In [11]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  

In [14]:
pip install textblob


  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [15]:
pip install pandas